From [Baker-Luecke] we get a whole infinite family of asymmetric L-space knots that all admit at least one alternating surgery. In this notebook we work with this infinite family. We build the 14 simplest of these knots and compute their dbc slopes.

In [1]:
# The relevant code.
import snappy
import time

def all_positive(manifold):
    '''
    Checks if the solution type of a triangulation is positive.
    '''
    return manifold.solution_type() == 'all tetrahedra positively oriented'

def find_positive_triangulations(manifold,number=1,tries=100):
    '''
    Searches for one triangulation with a positive solution type.
    (Or if number is set to a different value also for different such triangulations.)
    '''
    M = manifold.copy()
    pos_triangulations=[]
    for i in range(tries):
        if all_positive(M):
            pos_triangulations.append(M)
            if len(pos_triangulations)==number:
                return pos_triangulations
            break
        M.randomize()
    for d in M.dual_curves(max_segments=500):
        X = M.drill(d)
        X = X.filled_triangulation()
        X.dehn_fill((1,0),-1)
        for i in range(tries):
            if all_positive(X):
                pos_triangulations.append(X)
                if len(pos_triangulations)==number:
                    return pos_triangulations
                break
            X.randomize()

    # In the closed case, here is another trick.
    if all(not c for c in M.cusp_info('is_complete')):
        for i in range(tries):
            # Drills out a random edge
            X = M.__class__(M.filled_triangulation())
            if all_positive(X):
                pos_triangulations.append(X)
                if len(pos_triangulations)==number:
                    return pos_triangulations
            break
            M.randomize()
    return pos_triangulations

def better_verify_hyperbolicity(K,index=10):
    pos_triang=find_positive_triangulations(K,tries=index)
    for X in pos_triang:
        try:
            w=X.verify_hyperbolicity()[0]
            if w:
                return w
        except RuntimeError:
            pass
    return False

def better_symmetry_group(M,index=100,num_pos_triang=1):
    '''
    This function computes the symmetry group of the input manifold. 
    If the second entry is True it is proven to be the symmetry group.
    '''
    full=False
    S='unclear'
    try:
        S=M.symmetry_group()
        full=S.is_full_group()
        if full:
            return (S,full)
    except (ValueError, RuntimeError, snappy.SnapPeaFatalError):
        pass
    pos_triang=find_positive_triangulations(M,tries=index,number=num_pos_triang)
    if pos_triang==[]:
        randomizeCount=0
        while randomizeCount<index and full==False:
            try:
                S=M.symmetry_group()
                full=S.is_full_group()
                if full:
                    return (S,full)
                M.randomize()
                randomizeCount=randomizeCount+1
            except (ValueError, RuntimeError, snappy.SnapPeaFatalError):
                M.randomize()
                randomizeCount=randomizeCount+1
    for X in pos_triang:
        randomizeCount=0
        while randomizeCount<index and full==False:
            try:
                S=X.symmetry_group()
                full=S.is_full_group()
                if full:
                    return (S,full)
                X.randomize()
                randomizeCount=randomizeCount+1
            except (ValueError, RuntimeError, snappy.SnapPeaFatalError):
                X.randomize()
                randomizeCount=randomizeCount+1
    return (S,full)
    
    
def Bound(systole):
    '''
    Takes as input a systole and computes the bound from FPS19.
    '''
    B= math.sqrt(2*math.pi/systole +56)
    if 9.98>B:
        return 9.98
    else:
        return B+0.01

def better_systole(M,index):
    '''
    Computes the systole of a manifold. 
    It returns 'unclear' if SnapPy cannot compute the systole.
    '''
    w=False
    randomizeCount=0
    try:
        systole=M.dual_curves()[0].complete_length.real() 
        w=True
    except ValueError:
        pass
    except RuntimeError:
        pass
    except SnapPeaFatalError:
        pass
    except IndexError:
        pass
    if w==False:
        while randomizeCount<index and w==False:
            M.randomize()
            randomizeCount=randomizeCount+1
            try:
                systole=M.length_spectrum()[0].length.real() 
                w=True
            except ValueError:
                pass
            except RuntimeError:
                pass
            except snappy.SnapPeaFatalError:
                pass
            except IndexError:
                pass
    if w==True:
        return systole
    if w==False:
        return 'unclear' 

def compute_exceptional_symmetry_slopes(knot,index=10,slopes_restricted=False):
    '''
    Returns the exceptional symmetry slopes, the exceptional slopes and and the slopes where SnapPy could not compute its symmetry groups.
    '''
    K=snappy.Manifold(knot)
    systole=better_systole(K,index)
    if systole=='unclear':
        return [K.name(),'cannot compute systole']
    B=Bound(systole)
    for count in range(1,100):
        try:
            small_slopes=[s for s in K.short_slopes(length=B*math.sqrt(K.cusp_areas()[0]))[0] if s not in [(1,0)]] # The small hyperbolic slopes.
            break
        except:
            K.randomize()
    if slopes_restricted!=False:
        small_slopes=[s for s in small_slopes if slopes_restricted[0]<=s[0]/s[1]<=slopes_restricted[1]]
    
    exceptional_symmetry_slopes=[]
    smaller_groups=[]
    unclear_slopes=[]
    [S,full]=better_symmetry_group(K,index)
    if full!=True:
        return [K.name(),'cannot compute symmetry group']
    for s in small_slopes:
        K.dehn_fill(s)
        [T,w]=better_symmetry_group(K,index)
        if w!=True:
            unclear_slopes.append([s,T])
        else:
            if T.order()>S.order():
                exceptional_symmetry_slopes.append([s,T])
            if T.order()<S.order():
                smaller_groups.append([s,T])
    return [K.name(),S,len(small_slopes),exceptional_symmetry_slopes,smaller_groups,unclear_slopes]

def better_volume(M,index=100,try_hard=False):
    '''Computes the verified volume. Returns 0 if SnapPy could not do it.'''
    count=0
    while count<index:
        try:
            return M.volume(verified=True)
        except:
            M.randomize()
            count=count+1
    if try_hard==True:
        pos_triang=find_positive_triangulations(M,number=1,tries=index)
        for X in pos_triang:
            vol=better_volume(X,index)
            if vol!=0:
                return vol
    return 0

def symmetry_group_via_drill(K,index=100,try_hard=False):
    '''
    Computes the symmetry group by drilling short curves and using FPS.
    ''' 
    for c in K.dual_curves(max_segments=500):
        P=K.drill(c)
        F=P.filled_triangulation()
        try:
            systole=better_systole(F,index)
            B=Bound(systole)
            small_slopes=F.short_slopes(length=B*math.sqrt(F.cusp_areas()[0]))[0]
            if (1,0) not in small_slopes:
                S=F.symmetry_group()
                full=S.is_full_group()
                if full:
                    hom=K.homology()
                    vol=better_volume(K)
                    diff=True
                    for s in small_slopes:
                        F.dehn_fill(s)
                        if F.homology()==hom:
                            if better_volume(F).overlaps(vol):
                                diff=False
                                break
                    if diff:
                        return [S,full]
        except:
            pass
    if try_hard:
        for c in K.dual_curves(max_segments=500):
            P=K.drill(c)
            F=P.filled_triangulation()
            try:
                systole=better_systole(F,index=index)
                B=Bound(systole)
                small_slopes=F.short_slopes(length=B*math.sqrt(F.cusp_areas()[0]))[0]
                if (1,0) not in small_slopes:
                    (S,full)=better_symmetry_group(K,try_hard=True,index=index,num_pos_triang=index)
                    if full:
                        hom=K.homology()
                        vol=better_volume(K)
                        diff=True
                        for s in small_slopes:
                            F.dehn_fill(s)
                            if F.homology()==hom:
                                if better_volume(F).overlaps(vol):
                                    diff=False
                                    break
                        if diff:
                            return [S,full]
            except:
                pass
    return ['unclear',False]

def symmetry_group_via_drill2(K,index=100,try_hard=False):
    '''
    Computes the symmetry group by drilling short curves and using FPS.
    ''' 
    for c in K.dual_curves():
        P=K.drill(c)
        for d in P.dual_curves():
            Q=P.drill(d)
            F=Q.filled_triangulation()
            try:
                systole=better_systole(F,index)
                B=Bound(systole)
                small_slopes=F.short_slopes(length=B*math.sqrt(F.cusp_areas()[0]))
                if (1,0) not in small_slopes[0]:
                    if (1,0) not in small_slopes[1]:
                        S=F.symmetry_group()
                        full=S.is_full_group()
                        if full:
                            hom=K.homology()
                            vol=better_volume(K)
                            diff=True
                            for s0 in small_slopes[0]:
                                for s1 in small_slopes[1]:
                                    F.dehn_fill([s0,s1])
                                    if F.homology()==hom:
                                        if better_volume(F).overlaps(vol):
                                            diff=False
                                            break
                                if diff:
                                    return [S,full]
            except:
                pass
    
    return ['unclear',False]

In [2]:
L=snappy.Manifold('NewSurgLinkReducedReduced.txt')
index_set=[[1,1,1,1,0],[1,1,0,1,1],[1,1,1,2,0],[1,1,2,1,0],[1,2,1,1,0],[2,1,1,1,0],[1,1,1,1,1],[1,1,0,2,1],[1,2,0,1,1],[1,1,2,2,0],[1,2,1,2,0],[2,1,1,2,0],[1,1,0,1,2],[2,1,0,1,1]]
knots=[]
for (m,b1,a1,a2,a3) in index_set:
    p=a1*a2*a3+a1+a3;
    q=a2*a3+1;
    L.dehn_fill([(p,q),(m,1),(1,1),(-1,1),(1,1),(-1,1),(-1-a1,1),(-a3,1),(1,1),(1,m),(a2,1),(1,b1),(0,0),(-1,b1)]);
    K=L.filled_triangulation();  #useful to get snappy to find the hyp structure
    K.set_peripheral_curves( [ (-1,0), K.homological_longitude() ] , 0) #set meridians and longitude right
    K.set_name('BL_knot:'+str([m,b1,a1,a2,a3]))
    K.simplify()
    knots.append(K)

In [3]:
knots

[BL_knot:[1, 1, 1, 1, 0](0,0),
 BL_knot:[1, 1, 0, 1, 1](0,0),
 BL_knot:[1, 1, 1, 2, 0](0,0),
 BL_knot:[1, 1, 2, 1, 0](0,0),
 BL_knot:[1, 2, 1, 1, 0](0,0),
 BL_knot:[2, 1, 1, 1, 0](0,0),
 BL_knot:[1, 1, 1, 1, 1](0,0),
 BL_knot:[1, 1, 0, 2, 1](0,0),
 BL_knot:[1, 2, 0, 1, 1](0,0),
 BL_knot:[1, 1, 2, 2, 0](0,0),
 BL_knot:[1, 2, 1, 2, 0](0,0),
 BL_knot:[2, 1, 1, 2, 0](0,0),
 BL_knot:[1, 1, 0, 1, 2](0,0),
 BL_knot:[2, 1, 0, 1, 1](0,0)]

These are the first 14 knots from [Baker-Lucke]. We will first verify that these are aymmetric hyperbolic knots.

In [4]:
start_time = time.time()

for K in knots:
    print(K)
    print('The knot is hyperbolic:',better_verify_hyperbolicity(K))
    print('Symmetry group:',better_symmetry_group(K))
    print('-----------------------')
    
print("--- Time taken: %s seconds ---" % ((time.time() - start_time)))

BL_knot:[1, 1, 1, 1, 0](0,0)
The knot is hyperbolic: True
Symmetry group: (0, True)
-----------------------
BL_knot:[1, 1, 0, 1, 1](0,0)
The knot is hyperbolic: True
Symmetry group: (0, True)
-----------------------
BL_knot:[1, 1, 1, 2, 0](0,0)
The knot is hyperbolic: True
Symmetry group: (0, True)
-----------------------
BL_knot:[1, 1, 2, 1, 0](0,0)
The knot is hyperbolic: True
Symmetry group: (0, True)
-----------------------
BL_knot:[1, 2, 1, 1, 0](0,0)
The knot is hyperbolic: True
Symmetry group: (0, True)
-----------------------
BL_knot:[2, 1, 1, 1, 0](0,0)
The knot is hyperbolic: True
Symmetry group: (0, True)
-----------------------
BL_knot:[1, 1, 1, 1, 1](0,0)
The knot is hyperbolic: True
Symmetry group: (0, True)
-----------------------
BL_knot:[1, 1, 0, 2, 1](0,0)
The knot is hyperbolic: True
Symmetry group: (0, True)
-----------------------
BL_knot:[1, 2, 0, 1, 1](0,0)
The knot is hyperbolic: True
Symmetry group: (0, True)
-----------------------
BL_knot:[1, 1, 2, 2, 0](0,0)

Since the knots are asymmetric they have only finitely many slopes that yield manifolds which are not asymmetric. Since every double branched cover has an involution every dbc slope is symmetric. And thus there are only finitely many dbc slopes. Using the results from Futer-Purcel-Schleimer, we compute these finitely many symmetric slopes. These finitely many slope then contain all dbc slopes.

In [6]:
global_start_time = time.time()

EXCSYM=[]
for K in knots:
    start_time = time.time()
    exc_sym=compute_exceptional_symmetry_slopes(K)
    print(exc_sym)
    EXCSYM.append([K,exc_sym[3],exc_sym[5]])
    print("--- Time taken: %s Minutes ---" % ((time.time() - start_time)/60))
    print('------------')

print("--- Total time taken: %s hours ---" % ((time.time() - global_start_time)/3600))

['BL_knot:[1, 1, 1, 1, 0]', 0, 130, [[(271, 1), Z/2], [(272, 1), Z/2], [(273, 1), Z/2 + Z/2], [(543, 2), Z/2], [(2171, 8), Z/2]], [], [[(259, 1), unknown], [(260, 1), unknown], [(284, 1), unknown], [(805, 3), unknown], [(826, 3), unknown], [(1075, 4), unknown], [(1085, 4), unknown], [(1892, 7), unknown], [(2438, 9), unknown], [(2443, 9), unknown], [(2449, 9), unknown], [(2719, 10), unknown]]]
--- Time taken: 5.1782369693120325 Minutes ---
------------
['BL_knot:[1, 1, 0, 1, 1]', 0, 95, [[(469, 1), Z/2], [(470, 1), Z/2], [(471, 1), Z/2], [(472, 1), Z/2], [(1411, 3), Z/2]], [], []]
--- Time taken: 2.6497965693473815 Minutes ---
------------
['BL_knot:[1, 1, 1, 2, 0]', 0, 111, [[(415, 1), Z/2], [(416, 1), Z/2], [(417, 1), Z/2], [(831, 2), Z/2]], [], []]
--- Time taken: 0.6705979704856873 Minutes ---
------------
['BL_knot:[1, 1, 2, 1, 0]', 0, 199, [[(558, 1), Z/2], [(6120, 11), Z/2]], [], [[(541, 1), unknown], [(542, 1), unknown], [(544, 1), unknown], [(546, 1), unknown], [(548, 1), unkno

['BL_knot:[1, 1, 0, 1, 2]', 0, 93, [[(1142, 1), Z/2], [(1143, 1), Z/2], [(1144, 1), Z/2]], [], [[(2285, 2), unknown], [(3418, 3), unknown], [(3425, 3), unknown], [(4563, 4), unknown], [(4567, 4), unknown], [(4581, 4), unknown], [(5706, 5), unknown], [(5708, 5), unknown], [(6853, 6), unknown]]]
--- Time taken: 3.643601083755493 Minutes ---
------------
['BL_knot:[2, 1, 0, 1, 1]', 0, 97, [[(1066, 1), Z/2], [(1067, 1), Z/2], [(1068, 1), Z/2]], [], [[(8531, 8), unknown]]]
--- Time taken: 1.0497129003206889 Minutes ---
------------
--- Total time taken: 2.426750612192684 hours ---


In [7]:
EXCSYM_New=[]

global_start_time = time.time()

for [K,sym_exc,unclear] in EXCSYM:
    print([K,sym_exc,unclear])
    sym_exc_new=sym_exc.copy()
    asym=[]
    for s in unclear:
        K.dehn_fill(s[0])
        print(K)
        [T,full]=better_symmetry_group(K,index=10,num_pos_triang=10)
        print('symmetry group:',T,full)
        if full:
            if T.order()!=1:
                sym_exc_new.append([s[0],T])
            if T.order()==1:
                asym.append(s[0])
    K.dehn_fill((0,0))
    unclear_new=[x for x in unclear if x[0] not in [y[0] for y in sym_exc_new]+asym]
    EXCSYM_New.append([K,sym_exc_new,unclear_new])
    print([K,sym_exc_new,unclear_new])
    print('------------')

print("--- Time taken: %s hours ---" % ((time.time() - global_start_time)/3600))

[BL_knot:[1, 1, 1, 1, 0](0,0), [[(271, 1), Z/2], [(272, 1), Z/2], [(273, 1), Z/2 + Z/2], [(543, 2), Z/2], [(2171, 8), Z/2]], [[(259, 1), unknown], [(260, 1), unknown], [(284, 1), unknown], [(805, 3), unknown], [(826, 3), unknown], [(1075, 4), unknown], [(1085, 4), unknown], [(1892, 7), unknown], [(2438, 9), unknown], [(2443, 9), unknown], [(2449, 9), unknown], [(2719, 10), unknown]]]
BL_knot:[1, 1, 1, 1, 0](259,1)
symmetry group: 0 True
BL_knot:[1, 1, 1, 1, 0](260,1)
symmetry group: 0 True
BL_knot:[1, 1, 1, 1, 0](284,1)
symmetry group: 0 True
BL_knot:[1, 1, 1, 1, 0](805,3)
symmetry group: 0 True
BL_knot:[1, 1, 1, 1, 0](826,3)
symmetry group: 0 True
BL_knot:[1, 1, 1, 1, 0](1075,4)
symmetry group: 0 True
BL_knot:[1, 1, 1, 1, 0](1085,4)
symmetry group: 0 True
BL_knot:[1, 1, 1, 1, 0](1892,7)
symmetry group: 0 True
BL_knot:[1, 1, 1, 1, 0](2438,9)
symmetry group: 0 True
BL_knot:[1, 1, 1, 1, 0](2443,9)
symmetry group: 0 True
BL_knot:[1, 1, 1, 1, 0](2449,9)
symmetry group: 0 True
BL_knot:[1, 1

symmetry group: 0 True
BL_knot:[1, 1, 2, 1, 0](5563,10)
symmetry group: 0 True
BL_knot:[1, 1, 2, 1, 0](5567,10)
symmetry group: 0 True
BL_knot:[1, 1, 2, 1, 0](5569,10)
symmetry group: 0 True
BL_knot:[1, 1, 2, 1, 0](5571,10)
symmetry group: 0 True
BL_knot:[1, 1, 2, 1, 0](6114,11)
symmetry group: unknown False
BL_knot:[1, 1, 2, 1, 0](6115,11)
symmetry group: unknown False
BL_knot:[1, 1, 2, 1, 0](6117,11)
symmetry group: 0 True
BL_knot:[1, 1, 2, 1, 0](6119,11)
symmetry group: 0 True
BL_knot:[1, 1, 2, 1, 0](6121,11)
symmetry group: 0 True
BL_knot:[1, 1, 2, 1, 0](6122,11)
symmetry group: 0 True
BL_knot:[1, 1, 2, 1, 0](6123,11)
symmetry group: 0 True
BL_knot:[1, 1, 2, 1, 0](6124,11)
symmetry group: 0 True
BL_knot:[1, 1, 2, 1, 0](6125,11)
symmetry group: 0 True
BL_knot:[1, 1, 2, 1, 0](6126,11)
symmetry group: 0 True
BL_knot:[1, 1, 2, 1, 0](6128,11)
symmetry group: 0 True
BL_knot:[1, 1, 2, 1, 0](6129,11)
symmetry group: unknown False
BL_knot:[1, 1, 2, 1, 0](6130,11)
symmetry group: unknown Fal

symmetry group: 0 True
BL_knot:[2, 1, 1, 1, 0](5291,9)
symmetry group: 0 True
BL_knot:[2, 1, 1, 1, 0](5293,9)
symmetry group: 0 True
BL_knot:[2, 1, 1, 1, 0](5296,9)
symmetry group: 0 True
BL_knot:[2, 1, 1, 1, 0](5297,9)
symmetry group: 0 True
BL_knot:[2, 1, 1, 1, 0](5300,9)
symmetry group: 0 True
BL_knot:[2, 1, 1, 1, 0](5867,10)
symmetry group: unknown False
BL_knot:[2, 1, 1, 1, 0](5871,10)
symmetry group: 0 True
BL_knot:[2, 1, 1, 1, 0](5877,10)
symmetry group: 0 True
BL_knot:[2, 1, 1, 1, 0](5879,10)
symmetry group: 0 True
BL_knot:[2, 1, 1, 1, 0](5881,10)
symmetry group: 0 True
BL_knot:[2, 1, 1, 1, 0](5883,10)
symmetry group: 0 True
BL_knot:[2, 1, 1, 1, 0](5887,10)
symmetry group: unknown False
BL_knot:[2, 1, 1, 1, 0](6456,11)
symmetry group: unknown False
BL_knot:[2, 1, 1, 1, 0](6459,11)
symmetry group: 0 True
BL_knot:[2, 1, 1, 1, 0](6461,11)
symmetry group: 0 True
BL_knot:[2, 1, 1, 1, 0](6463,11)
symmetry group: 0 True
BL_knot:[2, 1, 1, 1, 0](6465,11)
symmetry group: 0 True
BL_knot:[

symmetry group: 0 True
BL_knot:[1, 2, 1, 2, 0](1697,2)
symmetry group: 0 True
BL_knot:[1, 2, 1, 2, 0](5910,7)
symmetry group: 0 True
BL_knot:[1, 2, 1, 2, 0](7591,9)
symmetry group: 0 True
BL_knot:[1, 2, 1, 2, 0](7595,9)
symmetry group: 0 True
[BL_knot:[1, 2, 1, 2, 0](0,0), [[(843, 1), Z/2], [(844, 1), Z/2]], []]
------------
[BL_knot:[2, 1, 1, 2, 0](0,0), [[(911, 1), Z/2], [(912, 1), Z/2]], [[(896, 1), unknown], [(897, 1), unknown], [(924, 1), unknown], [(926, 1), unknown], [(927, 1), unknown], [(1809, 2), unknown], [(1811, 2), unknown], [(1835, 2), unknown], [(2720, 3), unknown], [(2722, 3), unknown], [(2746, 3), unknown], [(2749, 3), unknown], [(3633, 4), unknown], [(3635, 4), unknown], [(3659, 4), unknown], [(3661, 4), unknown], [(4544, 5), unknown], [(4546, 5), unknown], [(4547, 5), unknown], [(4548, 5), unknown], [(4549, 5), unknown], [(4568, 5), unknown], [(4571, 5), unknown], [(4572, 5), unknown], [(5461, 6), unknown], [(5479, 6), unknown], [(5483, 6), unknown], [(6369, 7), unkn

In [8]:
EXCSYM=EXCSYM_New.copy()

EXCSYM_New=[]

global_start_time = time.time()

for [K,sym_exc,unclear] in EXCSYM:
    print([K,sym_exc,unclear])
    sym_exc_new=sym_exc.copy()
    asym=[]
    for s in unclear:
        K.dehn_fill(s[0])
        print(K)
        [T,full]=symmetry_group_via_drill2(K)
        print('symmetry group:',T,full)
        if full:
            if T.order()!=1:
                sym_exc_new.append([s[0],T])
            if T.order()==1:
                asym.append(s[0])
    K.dehn_fill((0,0))
    unclear_new=[x for x in unclear if x[0] not in [y[0] for y in sym_exc_new]+asym]
    EXCSYM_New.append([K,sym_exc_new,unclear_new])
    print([K,sym_exc_new,unclear_new])
    print('------------')

print("--- Time taken: %s hours ---" % ((time.time() - global_start_time)/3600))

[BL_knot:[1, 1, 1, 1, 0](0,0), [[(271, 1), Z/2], [(272, 1), Z/2], [(273, 1), Z/2 + Z/2], [(543, 2), Z/2], [(2171, 8), Z/2]], []]
[BL_knot:[1, 1, 1, 1, 0](0,0), [[(271, 1), Z/2], [(272, 1), Z/2], [(273, 1), Z/2 + Z/2], [(543, 2), Z/2], [(2171, 8), Z/2]], []]
------------
[BL_knot:[1, 1, 0, 1, 1](0,0), [[(469, 1), Z/2], [(470, 1), Z/2], [(471, 1), Z/2], [(472, 1), Z/2], [(1411, 3), Z/2]], []]
[BL_knot:[1, 1, 0, 1, 1](0,0), [[(469, 1), Z/2], [(470, 1), Z/2], [(471, 1), Z/2], [(472, 1), Z/2], [(1411, 3), Z/2]], []]
------------
[BL_knot:[1, 1, 1, 2, 0](0,0), [[(415, 1), Z/2], [(416, 1), Z/2], [(417, 1), Z/2], [(831, 2), Z/2]], []]
[BL_knot:[1, 1, 1, 2, 0](0,0), [[(415, 1), Z/2], [(416, 1), Z/2], [(417, 1), Z/2], [(831, 2), Z/2]], []]
------------
[BL_knot:[1, 1, 2, 1, 0](0,0), [[(558, 1), Z/2], [(6120, 11), Z/2]], [[(541, 1), unknown], [(556, 1), at least Z/2], [(557, 1), at least Z/2], [(572, 1), unknown], [(1113, 2), unknown], [(1129, 2), unknown], [(1655, 3), unknown], [(1684, 3), unkno

symmetry group: unclear False
BL_knot:[1, 1, 2, 2, 0](832,1)
symmetry group: unclear False
BL_knot:[1, 1, 2, 2, 0](860,1)
symmetry group: unclear False
BL_knot:[1, 1, 2, 2, 0](1677,2)
symmetry group: unclear False
BL_knot:[1, 1, 2, 2, 0](1705,2)
symmetry group: unclear False
BL_knot:[1, 1, 2, 2, 0](2522,3)
symmetry group: unclear False
BL_knot:[1, 1, 2, 2, 0](2551,3)
symmetry group: unclear False
BL_knot:[1, 1, 2, 2, 0](4241,5)
symmetry group: unclear False
BL_knot:[1, 1, 2, 2, 0](5907,7)
symmetry group: unclear False
BL_knot:[1, 1, 2, 2, 0](5930,7)
symmetry group: unclear False
BL_knot:[1, 1, 2, 2, 0](6775,8)
symmetry group: unclear False
BL_knot:[1, 1, 2, 2, 0](7601,9)
symmetry group: unclear False
BL_knot:[1, 1, 2, 2, 0](7618,9)
symmetry group: unclear False
BL_knot:[1, 1, 2, 2, 0](8449,10)
symmetry group: unclear False
BL_knot:[1, 1, 2, 2, 0](8461,10)
symmetry group: unclear False
BL_knot:[1, 1, 2, 2, 0](9299,11)
symmetry group: unclear False
BL_knot:[1, 1, 2, 2, 0](9300,11)
symmet

In [9]:
EXCSYM=EXCSYM_New.copy()

EXCSYM_New=[]

global_start_time = time.time()

for [K,sym_exc,unclear] in EXCSYM:
    print([K,sym_exc,unclear])
    sym_exc_new=sym_exc.copy()
    asym=[]
    for s in unclear:
        K.dehn_fill(s[0])
        print(K)
        [T,full]=better_symmetry_group(K,index=10,num_pos_triang=10)
        print('symmetry group:',T,full)
        if full:
            if T.order()!=1:
                sym_exc_new.append([s[0],T])
            if T.order()==1:
                asym.append(s[0])
    K.dehn_fill((0,0))
    unclear_new=[x for x in unclear if x[0] not in [y[0] for y in sym_exc_new]+asym]
    EXCSYM_New.append([K,sym_exc_new,unclear_new])
    print([K,sym_exc_new,unclear_new])
    print('------------')

print("--- Time taken: %s hours ---" % ((time.time() - global_start_time)/3600))

[BL_knot:[1, 1, 1, 1, 0](0,0), [[(271, 1), Z/2], [(272, 1), Z/2], [(273, 1), Z/2 + Z/2], [(543, 2), Z/2], [(2171, 8), Z/2]], []]
[BL_knot:[1, 1, 1, 1, 0](0,0), [[(271, 1), Z/2], [(272, 1), Z/2], [(273, 1), Z/2 + Z/2], [(543, 2), Z/2], [(2171, 8), Z/2]], []]
------------
[BL_knot:[1, 1, 0, 1, 1](0,0), [[(469, 1), Z/2], [(470, 1), Z/2], [(471, 1), Z/2], [(472, 1), Z/2], [(1411, 3), Z/2]], []]
[BL_knot:[1, 1, 0, 1, 1](0,0), [[(469, 1), Z/2], [(470, 1), Z/2], [(471, 1), Z/2], [(472, 1), Z/2], [(1411, 3), Z/2]], []]
------------
[BL_knot:[1, 1, 1, 2, 0](0,0), [[(415, 1), Z/2], [(416, 1), Z/2], [(417, 1), Z/2], [(831, 2), Z/2]], []]
[BL_knot:[1, 1, 1, 2, 0](0,0), [[(415, 1), Z/2], [(416, 1), Z/2], [(417, 1), Z/2], [(831, 2), Z/2]], []]
------------
[BL_knot:[1, 1, 2, 1, 0](0,0), [[(558, 1), Z/2], [(6120, 11), Z/2]], [[(541, 1), unknown], [(556, 1), at least Z/2], [(557, 1), at least Z/2], [(572, 1), unknown], [(1113, 2), unknown], [(1129, 2), unknown], [(1655, 3), unknown], [(1684, 3), unkno

symmetry group: unknown False
BL_knot:[1, 1, 2, 2, 0](832,1)
symmetry group: 0 True
BL_knot:[1, 1, 2, 2, 0](860,1)
symmetry group: unknown False
BL_knot:[1, 1, 2, 2, 0](1677,2)
symmetry group: unknown False
BL_knot:[1, 1, 2, 2, 0](1705,2)
symmetry group: unknown False
BL_knot:[1, 1, 2, 2, 0](2522,3)
symmetry group: unknown False
BL_knot:[1, 1, 2, 2, 0](2551,3)
symmetry group: unknown False
BL_knot:[1, 1, 2, 2, 0](4241,5)
symmetry group: unknown False
BL_knot:[1, 1, 2, 2, 0](5907,7)
symmetry group: unknown False
BL_knot:[1, 1, 2, 2, 0](5930,7)
symmetry group: unknown False
BL_knot:[1, 1, 2, 2, 0](6775,8)
symmetry group: unknown False
BL_knot:[1, 1, 2, 2, 0](7601,9)
symmetry group: unknown False
BL_knot:[1, 1, 2, 2, 0](7618,9)
symmetry group: unknown False
BL_knot:[1, 1, 2, 2, 0](8449,10)
symmetry group: unknown False
BL_knot:[1, 1, 2, 2, 0](8461,10)
symmetry group: unknown False
BL_knot:[1, 1, 2, 2, 0](9299,11)
symmetry group: unknown False
BL_knot:[1, 1, 2, 2, 0](9300,11)
symmetry grou

In [10]:
EXCSYM=EXCSYM_New.copy()

EXCSYM

[[BL_knot:[1, 1, 1, 1, 0](0,0),
  [[(271, 1), Z/2],
   [(272, 1), Z/2],
   [(273, 1), Z/2 + Z/2],
   [(543, 2), Z/2],
   [(2171, 8), Z/2]],
  []],
 [BL_knot:[1, 1, 0, 1, 1](0,0),
  [[(469, 1), Z/2],
   [(470, 1), Z/2],
   [(471, 1), Z/2],
   [(472, 1), Z/2],
   [(1411, 3), Z/2]],
  []],
 [BL_knot:[1, 1, 1, 2, 0](0,0),
  [[(415, 1), Z/2], [(416, 1), Z/2], [(417, 1), Z/2], [(831, 2), Z/2]],
  []],
 [BL_knot:[1, 1, 2, 1, 0](0,0),
  [[(558, 1), Z/2], [(6120, 11), Z/2]],
  [[(541, 1), unknown],
   [(556, 1), at least Z/2],
   [(557, 1), at least Z/2],
   [(572, 1), unknown],
   [(1113, 2), unknown],
   [(1129, 2), unknown],
   [(1655, 3), unknown],
   [(1684, 3), unknown],
   [(1685, 3), unknown],
   [(2241, 4), unknown],
   [(2769, 5), unknown],
   [(2797, 5), unknown],
   [(3353, 6), unknown],
   [(3883, 7), unknown],
   [(3884, 7), unknown],
   [(3909, 7), unknown],
   [(4441, 8), unknown],
   [(4465, 8), unknown],
   [(4999, 9), unknown],
   [(5020, 9), unknown],
   [(5021, 9), unknown]

In [11]:
def all_positive(manifold):
    '''
    Checks if the solution type of a triangulation is positive.
    '''
    return manifold.solution_type() == 'all tetrahedra positively oriented'

def find_positive_triangulations(manifold,number=1,tries=100):
    '''
    Searches for one triangulation with a positive solution type.
    (Or if number is set to a different value also for different such triangulations.)
    '''
    M = manifold.copy()
    pos_triangulations=[]
    for i in range(tries):
        if all_positive(M):
            pos_triangulations.append(M)
            if len(pos_triangulations)==number:
                return pos_triangulations
            break
        M.randomize()
    for d in M.dual_curves(max_segments=500):
        X = M.drill(d)
        X = X.filled_triangulation()
        X.dehn_fill((1,0),-1)
        for i in range(tries):
            if all_positive(X):
                pos_triangulations.append(X)
                if len(pos_triangulations)==number:
                    return pos_triangulations
                break
            X.randomize()

    # In the closed case, here is another trick.
    if all(not c for c in M.cusp_info('is_complete')):
        for i in range(tries):
            # Drills out a random edge
            X = M.__class__(M.filled_triangulation())
            if all_positive(X):
                pos_triangulations.append(X)
                if len(pos_triangulations)==number:
                    return pos_triangulations
            break
            M.randomize()
    return pos_triangulations

def better_symmetry_group(M,index=100,num_pos_triang=1):
    '''
    This function computes the symmetry group of the input manifold. 
    If the second entry is True it is proven to be the symmetry group.
    '''
    full=False
    S='unclear'
    try:
        S=M.symmetry_group()
        full=S.is_full_group()
        if full:
            return (S,full)
    except (ValueError, RuntimeError, snappy.SnapPeaFatalError):
        pass
    pos_triang=find_positive_triangulations(M,tries=index,number=num_pos_triang)
    if pos_triang==[]:
        randomizeCount=0
        while randomizeCount<index and full==False:
            try:
                S=M.symmetry_group()
                full=S.is_full_group()
                if full:
                    return (S,full)
                M.randomize()
                randomizeCount=randomizeCount+1
            except (ValueError, RuntimeError, snappy.SnapPeaFatalError):
                M.randomize()
                randomizeCount=randomizeCount+1
    for X in pos_triang:
        randomizeCount=0
        while randomizeCount<index and full==False:
            try:
                S=X.symmetry_group()
                full=S.is_full_group()
                if full:
                    return (S,full)
                X.randomize()
                randomizeCount=randomizeCount+1
            except (ValueError, RuntimeError, snappy.SnapPeaFatalError):
                X.randomize()
                randomizeCount=randomizeCount+1
    return (S,full)

def better_systole_word(M,index=100,length=1):
    '''
    Computes the word of the systole of a manifold. 
    It returns False if SnapPy cannot compute the systole.
    '''
    randomizeCount=0
    while randomizeCount<index:
        try:
            return M.length_spectrum(cutoff=length,include_words=True)[0].word
        except:
            M.randomize()
            randomizeCount=randomizeCount+1
    return False

def symmetry_group_via_drill(mfd,high_precision=False,maxlength=1,tries=10,try_hard=True):
    '''
    Computes the length spectrum of a given input manifold M up to a given maximal length, 
    drills out the shortest geodesic, and computes the symmetry group of that drilled manifold.
    Since every symmetry of M sends the shortest geodesic to itself, this is the symmetry group of M. 
    '''
    M=snappy.Manifold(mfd)
    if high_precision:
        M=M.high_precision()
    for i in range(tries):
        try:
            systole=better_systole_word(M,length=maxlength)
            P=M.drill_word(systole)
            return better_symmetry_group(P)
        except:
            M.randomize()
    if try_hard:
        for c in K.dual_curves():
            P=K.drill(c)
            P=P.filled_triangulation()
            P.dehn_fill((1,0))
            for i in range(3):
                try:
                    systole=better_systole_word(P,length=maxlength)
                    X=P.drill_word(systole)
                    return better_symmetry_group(X)
                except:
                    P.randomize()
    return False

In [12]:
EXCSYM_New=[]

global_start_time = time.time()

for [K,sym_exc,unclear] in EXCSYM:
    print([K,sym_exc,unclear])
    sym_exc_new=sym_exc.copy()
    asym=[]
    for s in unclear:
        K.dehn_fill(s[0])
        print(K)
        [T,full]=symmetry_group_via_drill(K,high_precision=False,maxlength=1,tries=25)
        print('symmetry group:',T,full)
        if full:
            if T.order()!=1:
                sym_exc_new.append([s[0],T])
            if T.order()==1:
                asym.append(s[0])
    K.dehn_fill((0,0))
    unclear_new=[x for x in unclear if x[0] not in [y[0] for y in sym_exc_new]+asym]
    EXCSYM_New.append([K,sym_exc_new,unclear_new])
    print([K,sym_exc_new,unclear_new])
    print('------------')
    
EXCSYM=EXCSYM_New.copy()

print("--- Time taken: %s hours ---" % ((time.time() - global_start_time)/3600))



[BL_knot:[1, 1, 1, 1, 0](0,0), [[(271, 1), Z/2], [(272, 1), Z/2], [(273, 1), Z/2 + Z/2], [(543, 2), Z/2], [(2171, 8), Z/2]], []]
[BL_knot:[1, 1, 1, 1, 0](0,0), [[(271, 1), Z/2], [(272, 1), Z/2], [(273, 1), Z/2 + Z/2], [(543, 2), Z/2], [(2171, 8), Z/2]], []]
------------
[BL_knot:[1, 1, 0, 1, 1](0,0), [[(469, 1), Z/2], [(470, 1), Z/2], [(471, 1), Z/2], [(472, 1), Z/2], [(1411, 3), Z/2]], []]
[BL_knot:[1, 1, 0, 1, 1](0,0), [[(469, 1), Z/2], [(470, 1), Z/2], [(471, 1), Z/2], [(472, 1), Z/2], [(1411, 3), Z/2]], []]
------------
[BL_knot:[1, 1, 1, 2, 0](0,0), [[(415, 1), Z/2], [(416, 1), Z/2], [(417, 1), Z/2], [(831, 2), Z/2]], []]
[BL_knot:[1, 1, 1, 2, 0](0,0), [[(415, 1), Z/2], [(416, 1), Z/2], [(417, 1), Z/2], [(831, 2), Z/2]], []]
------------
[BL_knot:[1, 1, 2, 1, 0](0,0), [[(558, 1), Z/2], [(6120, 11), Z/2]], [[(541, 1), unknown], [(556, 1), at least Z/2], [(557, 1), at least Z/2], [(572, 1), unknown], [(1113, 2), unknown], [(1129, 2), unknown], [(1655, 3), unknown], [(1684, 3), unkno

symmetry group: 0 True
BL_knot:[2, 1, 1, 2, 0](1809,2)
symmetry group: 0 True
BL_knot:[2, 1, 1, 2, 0](2720,3)
symmetry group: 0 True
BL_knot:[2, 1, 1, 2, 0](2749,3)
symmetry group: 0 True
BL_knot:[2, 1, 1, 2, 0](3633,4)
symmetry group: 0 True
BL_knot:[2, 1, 1, 2, 0](3661,4)
symmetry group: 0 True
BL_knot:[2, 1, 1, 2, 0](4544,5)
symmetry group: 0 True
BL_knot:[2, 1, 1, 2, 0](4571,5)
symmetry group: 0 True
BL_knot:[2, 1, 1, 2, 0](4572,5)
symmetry group: 0 True
BL_knot:[2, 1, 1, 2, 0](5483,6)
symmetry group: 0 True
BL_knot:[2, 1, 1, 2, 0](6369,7)
symmetry group: 0 True
BL_knot:[2, 1, 1, 2, 0](6393,7)
symmetry group: 0 True
BL_knot:[2, 1, 1, 2, 0](7283,8)
symmetry group: 0 True
BL_knot:[2, 1, 1, 2, 0](7303,8)
symmetry group: 0 True
BL_knot:[2, 1, 1, 2, 0](8195,9)
symmetry group: 0 True
BL_knot:[2, 1, 1, 2, 0](8213,9)
symmetry group: 0 True
BL_knot:[2, 1, 1, 2, 0](9109,10)
symmetry group: 0 True
BL_knot:[2, 1, 1, 2, 0](9123,10)
symmetry group: 0 True
BL_knot:[2, 1, 1, 2, 0](10022,11)
symmet

In [13]:
EXCSYM

[[BL_knot:[1, 1, 1, 1, 0](0,0),
  [[(271, 1), Z/2],
   [(272, 1), Z/2],
   [(273, 1), Z/2 + Z/2],
   [(543, 2), Z/2],
   [(2171, 8), Z/2]],
  []],
 [BL_knot:[1, 1, 0, 1, 1](0,0),
  [[(469, 1), Z/2],
   [(470, 1), Z/2],
   [(471, 1), Z/2],
   [(472, 1), Z/2],
   [(1411, 3), Z/2]],
  []],
 [BL_knot:[1, 1, 1, 2, 0](0,0),
  [[(415, 1), Z/2], [(416, 1), Z/2], [(417, 1), Z/2], [(831, 2), Z/2]],
  []],
 [BL_knot:[1, 1, 2, 1, 0](0,0),
  [[(558, 1), Z/2], [(6120, 11), Z/2], [(556, 1), Z/2], [(557, 1), Z/2]],
  []],
 [BL_knot:[1, 2, 1, 1, 0](0,0), [[(554, 1), Z/2], [(555, 1), Z/2]], []],
 [BL_knot:[2, 1, 1, 1, 0](0,0),
  [[(589, 1), Z/2], [(6460, 11), Z/2], [(587, 1), Z/2], [(588, 1), Z/2]],
  []],
 [BL_knot:[1, 1, 1, 1, 1](0,0),
  [[(1155, 1), Z/2], [(1156, 1), Z/2], [(5778, 5), Z/2]],
  []],
 [BL_knot:[1, 1, 0, 2, 1](0,0),
  [[(1008, 1), Z/2], [(1009, 1), Z/2], [(1010, 1), Z/2]],
  []],
 [BL_knot:[1, 2, 0, 1, 1](0,0), [[(965, 1), Z/2], [(966, 1), Z/2]], []],
 [BL_knot:[1, 1, 2, 2, 0](0,0), [[(

This finishes the classification of the symetry-exceptional slopes. Next, we search for explicit branching sets.

In [15]:
import time
import snappy
import pandas
DBChomologies = pandas.read_csv("DBChomologies_non_alternating.csv") # We load a list of all non alternating links in 
#the HTLinkExteriors together with their homologies (if the homology is cyclic) and their crossing numbers.
### REMARK: To build the list we need to run the code in the file Build_DBChomologies which takes around 60 minutes. 
#Here we just load it.


def double_branched_cover(link):
    """
    Returns the double branched cover of the link.
    """
    L=link.copy()
    for i in range(L.num_cusps()):
        L.dehn_fill((2,0),i)
    for cov in L.covers(2):
        if (2.0, 0.0) not in cov.cusp_info('filling'):
            return cov
        


def better_is_isometric_to(X,Y,index):
    """
    Returns True if X and Y are isometric.
    Returns False if X and Y have different homologies. TO DO: Use volume to rigorously distinguish X and Y.
    Returns 'unclear' if SnapPy cannot verify it.
    The higher the index the harder SnapPy tries.
    """     
    w='unclear'
    if X.homology()!=Y.homology():
        w=False
    if w=='unclear':
        for i in (0,index):
            try:
                w=X.is_isometric_to(Y)
            except RuntimeError:
                pass
            except snappy.SnapPeaFatalError:
                pass
            if w==True:
                break
            if w==False:
                w='unclear'
            X.randomize()
            Y.randomize()
            i=i+1
    return w

def possible_DBC(homologies,max_crossings=15):
    """
    Takes a list of orders of homologies and returns a list consisting of all DBC of alternating links in the HT link table with that homologies together with the link names.
    """
    DBCList=[]
    LINKS=[]
    for order in homologies:
        LINKS=LINKS+DBChomologies.loc[(DBChomologies['homology']==order) & (DBChomologies['crossings']<=max_crossings)]['knot'].tolist()
    for link in LINKS:
        L=snappy.Manifold(link)
        D=double_branched_cover(L)
        DBCList.append([D,link])
    return DBCList

### The following two functions are written by Dunfield and search for positive triangulations.

def all_positive(manifold):
    return manifold.solution_type() == 'all tetrahedra positively oriented'

def find_positive_triangulation(manifold, tries=100):
    M = manifold.copy()
    for i in range(tries):
        if all_positive(M):
            return M
        M.randomize()
    for d in M.dual_curves():
        X = M.drill(d)
        X = X.filled_triangulation()
        X.dehn_fill((1,0))
        for i in range(tries):
            if all_positive(X):
                return X
            X.randomize()

    # In the closed case, here is another trick.
    if all(not c for c in M.cusp_info('is_complete')):
        for i in range(tries):
            # Drills out a random edge
            X = M.__class__(M.filled_triangulation())
            if all_positive(X):
                return X
            M.randomize()
            
def better_find_positive_triangulation(M,tries=1):
    '''
    Search for a positive triangulation, but ignores errors.
    '''
    RandomizeCount=0
    while RandomizeCount<tries:
        try:
            X=find_positive_triangulation(M)
            return X
        except snappy.SnapPeaFatalError:
            M.randomize()
            RandomizeCount=RandomizeCount+1
    return None

def is_alternating(knot,slope,try_hard=False,index=10,tries=1,max_cro=15):
    '''
    Checks if the slope is alternating.
    '''
    K=snappy.Manifold(knot)
    K.dehn_fill(slope)
    DBC=possible_DBC([K.homology().order()],max_crossings=max_cro) 
    for D in DBC:
        w=better_is_isometric_to(D[0],K,index)
        if w==True:
            return [slope,D[1]]
    if try_hard:
        X=better_find_positive_triangulation(K,tries)
        if X is not None:
            for D in DBC:
                Y=better_find_positive_triangulation(D[0],tries)
                if Y is not None:
                    w=better_is_isometric_to(X,Y,index)
                    if w==True:
                        return [slope,D[1]]
    return False

In [17]:
start_time = time.time()
branching_sets=[]
for [K,dbcslopes,unclear] in EXCSYM:
    for s in dbcslopes:
        w=is_alternating(K,s[0],index=25,max_cro=15)
        if w!=False:
            branching_sets.append([K,w[0],w[1]])
            print('We found a branching set:',knot,s,w[1])
        
print('Total number of branching sets we have found:', len(branching_sets))
print('Total time taken: %s minutes ---' % ((time.time() - start_time)/60))

Total number of branching sets we have found: 0
Total time taken: 0.8718295176823934 minutes ---


In [24]:
DBChomologies_branching = pandas.read_csv("DBChomologies_one_filling.csv")  
### REMARK: To build the list we need to run the code in the file Build_DBChomologies which takes around 3 days. Here we just load it.


def double_branched_cover(link):
    """
    Returns the double branched covers of the link. This works also for links in a more general manifold. 
    Note that a knot in a more general manifold may have more than one double branched cover 
    (or no double branched cover at all if the knot represents a primitive element in homology).
    This function will return the complete list of all double branched covers of the link.
    """
    L=link.copy()
    for i in range(L.num_cusps()):
        if L.cusp_info(i).filling==(0.0, 0.0):
            L.dehn_fill((2,0),i)
    return [cov for cov in L.covers(2) if (2.0, 0.0) not in cov.cusp_info('filling')]

def possible_DBC_surgery_diagrams(homologies,max_crossings=15):
    """
    Reads off the possible surgery diagrams.
    """
    DBCList=[]
    LINKS=[]
    CUSPS=[]
    SLOPES_STRINGS=[]
    for order in homologies:
        LINKS=LINKS+DBChomologies_branching.loc[(DBChomologies_branching['homology']==order) & (DBChomologies_branching['crossings']<=max_crossings)]['knot'].tolist()
        CUSPS=CUSPS+DBChomologies_branching.loc[(DBChomologies_branching['homology']==order) & (DBChomologies_branching['crossings']<=max_crossings)]['cusp'].tolist()
        SLOPES_STRINGS=SLOPES_STRINGS+DBChomologies_branching.loc[(DBChomologies_branching['homology']==order) & (DBChomologies_branching['crossings']<=max_crossings)]['filling'].tolist()
    SLOPES=[]
    for string in SLOPES_STRINGS:
        string_without_brackets=string[1:-1]
        SLOPES.append(tuple(map(int, string_without_brackets.split(', '))))
    for i in range(0,len(LINKS)):
        DBCList.append([LINKS[i],SLOPES[i],CUSPS[i]])
    return DBCList

def search_for_branching_set(knot,slope,try_hard=False,index=10,tries=1,max_crossings=15):
    '''
    Searchs for a surgery diagram of the rbanching set
    '''  
    K=snappy.Manifold(knot)
    K.dehn_fill(slope)
    DBC=possible_DBC_surgery_diagrams([K.homology().order()],max_crossings) 
    for DIAGRAM in DBC:
        L=snappy.Manifold(DIAGRAM[0])
        L.dehn_fill(DIAGRAM[1],DIAGRAM[2])
        for D in double_branched_cover(L):
            w=better_is_isometric_to(D,K,index)
            if w==True:
                return [knot,slope,DIAGRAM[0],DIAGRAM[1],DIAGRAM[2]]
    if try_hard:
        X=better_find_positive_triangulation(K,tries)
        if X is not None:
            for DIAGRAM in DBC:
                L=snappy.Manifold(DIAGRAM[0])
                L.dehn_fill(DIAGRAM[1],DIAGRAM[2])
                for D in double_branched_cover(L):
                    Y=better_find_positive_triangulation(D,tries)
                    if Y is not None:
                        w=better_is_isometric_to(X,Y,index)
                        if w==True:
                            return [knot,slope,DIAGRAM[0],DIAGRAM[1],DIAGRAM[2]]                                  
    return False

def search_for_three_branching_sets(knot,slope,try_hard=False,index=10,tries=1,max_crossings=15):
    '''
    Searchs for a surgery diagram of the branching set
    '''  
    BRANCHING_SETS=[]
    K=snappy.Manifold(knot)
    K.dehn_fill(slope)
    homologies=[]
    DBC=possible_DBC_surgery_diagrams([K.homology().order()],max_crossings) 
    for DIAGRAM in DBC:
        L=snappy.Manifold(DIAGRAM[0])
        L.dehn_fill(DIAGRAM[1],DIAGRAM[2])
        for D in double_branched_cover(L):
            w=better_is_isometric_to(D,K,index)
            if w==True:
                BRANCHING_SETS=BRANCHING_SETS+[[knot,slope,DIAGRAM[0],DIAGRAM[1],DIAGRAM[2]]]
                for i in range(L.num_cusps()):
                    if L.cusp_info(i).filling==(0.0, 0.0):
                        L.dehn_fill((1,0),i)
                if L.homology().order() not in homologies:
                    homologies.append(L.homology().order())
                if len(homologies)==3:
                    return BRANCHING_SETS
    if try_hard:
        X=better_find_positive_triangulation(K,tries)
        if X is not None:
            for DIAGRAM in DBC:
                L=snappy.Manifold(DIAGRAM[0])
                L.dehn_fill(DIAGRAM[1],DIAGRAM[2])
                for D in double_branched_cover(L):
                    Y=better_find_positive_triangulation(D,tries)
                    if Y is not None:
                        w=better_is_isometric_to(X,Y,index)
                        if w==True:
                            BRANCHING_SETS=BRANCHING_SETS+[[knot,slope,DIAGRAM[0],DIAGRAM[1],DIAGRAM[2]]]
                            for i in range(L.num_cusps()):
                                if L.cusp_info(i).filling==(0.0, 0.0):
                                    L.dehn_fill((1,0),i)
                            if L.homology().order() not in homologies:
                                homologies.append(L.homology().order())
                            if len(homologies)==3:
                                return BRANCHING_SETS
    return BRANCHING_SETS

In [30]:
start_time = time.time()
branching_sets=[]
for [K,dbcslopes,unclear] in EXCSYM:
    for s in dbcslopes:
        w=search_for_branching_set(K,s[0])
        if w!=False:
            branching_sets.append(w)
            print('We found a branching set:',w)
        
print('Total number of branching sets we have found:', len(branching_sets))
print('Total time taken: %s minutes ---' % ((time.time() - start_time)/60))

We found a branching set: [BL_knot:[1, 1, 1, 1, 0](0,0), (273, 1), 'L13n3686', (3, 1), 0]
We found a branching set: [BL_knot:[1, 1, 1, 1, 0](0,0), (543, 2), 'L13a2543', (3, 1), 0]
We found a branching set: [BL_knot:[1, 1, 0, 1, 1](0,0), (472, 1), 'L14n55138', (4, 1), 1]
We found a branching set: [BL_knot:[1, 1, 1, 2, 0](0,0), (417, 1), 'L14n16219', (3, 1), 0]
We found a branching set: [BL_knot:[1, 1, 1, 2, 0](0,0), (831, 2), 'L14a9663', (-3, 1), 0]
Total number of branching sets we have found: 5
Total time taken: 26.212074466546376 minutes ---


In [31]:
K=EXCSYM[0][0]
K

BL_knot:[1, 1, 1, 1, 0](0,0)

In [32]:
w=search_for_three_branching_sets(K,(273, 1),try_hard=False,index=10,tries=1,max_crossings=15)
w

[[BL_knot:[1, 1, 1, 1, 0](0,0), (273, 1), 'L13n3686', (3, 1), 0]]